# 1. Initialisation

## 1.1. File System

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

! chmod -R 777 'drive/My Drive/Otago/F. The Thesis/Source Code'
 
% cd 'drive/My Drive/Otago/F. The Thesis/Source Code'

Mounted at /content/drive
/content/drive/My Drive/Otago/F. The Thesis/Source Code


In [ ]:
!pip install livelossplot
!pip install textaugment
!pip install keras_lr_multiplier

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 1.2 MB 17.9 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 2.8 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15734 sha256=031eb3168e18e3b4900d88bfc503582f55e46ea631bf0feac40c9c3c7adbf4ee
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
     |████████████████████████████████| 312 kB 8.5 MB/s 
     |████████████████████████████████| 2.9 MB 77.7 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
  Created wheel for keras-lr-multiplier: filename=keras_lr_multiplier-0.9.0-py3-none-any.whl size=5465 sha256=f932db566103ad84e8f286fdf07f0a81689e82d189a12685d0719a89176ed555
  Stored in directory: /root/.cache/pip/wheels/08/06/a6/7992ebb2bf96326b34

## 1.2. Importing Libraries

In [ ]:
import pandas as pd
import time
import re
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import nltk
from livelossplot import PlotLossesKeras
from sklearn.model_selection import train_test_split
import random
import csv
import pickle
import string

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import tensorflow_hub as hub
from keras import Model
from keras import losses
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras import Input
from keras.layers import Dense, Concatenate, BatchNormalization
from keras.layers import Flatten, Dot, Reshape
from keras.layers import LSTM, GRU, Bidirectional, Conv1D, Dropout
from keras.layers import Activation
from keras.layers.embeddings import Embedding
from keras.backend.tensorflow_backend import set_session
import keras.backend as K
from tensorflow.keras import regularizers
from keras.layers.merge import concatenate
from keras.losses import mse
from keras_lr_multiplier import LRMultiplier

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

In [ ]:
from textaugment import Wordnet

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
sys.path.insert(0, './Libraries')
import preprocess

## 1.3. GPU Initialisation

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5



## 1.4. Global Variables

In [ ]:
model_name = 'RIVEC300_SMALL_'
embed_size = 300
padding_len = 100
vocab_size = 10000
Use_Negative = True

# 2. Data Preparation

## 2.1. Input and Output

In [ ]:
def clean_text(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [ ]:
normInputFile = f'./Inputs/SEmHus_Enriched_Normalised_Dataset.csv'
normDF = pd.read_csv(normInputFile)
normDF = normDF.sample(n = len(normDF))

In [ ]:
len(normDF)

63609

In [ ]:
train_limit = int(len(normDF) * 0.8)
train_split = int(train_limit * 0.8)
valid_split = train_limit - train_split

In [ ]:
print(train_limit)

50887


In [ ]:
file_name = f'./Inputs/{model_name}SEmHus_Enriched_Normalised_Dataset_Train.csv'
trainDF = pd.read_csv(file_name)

In [ ]:
file_name = f'./Inputs/{model_name}SEmHus_Enriched_Normalised_Dataset_Test.csv'
trainDF = pd.read_csv(file_name)

In [ ]:
docs = normDF['text'].tolist()

labActor = [eval(v) for v in normDF['source_name'].tolist()]
labAgency = [eval(v) for v in normDF['source_type'].tolist()]
labSector = [eval(v) for v in normDF['theme'].tolist()]
labPlace = [eval(v) for v in normDF['country_name'].tolist()]
labYear = [eval(v) for v in normDF['year_created'].tolist()]
labMonth = [eval(v) for v in normDF['month_created'].tolist()]
labReason = [eval(v) for v in normDF['disaster_type'].tolist()]
labGoal = [eval(v) for v in normDF['development_goal'].tolist()]

In [ ]:
docs = [' '.join(str(d).lower().split()) for d in docs]

In [ ]:
with open('./Dictionary/Extended_Stop_Words.txt', 'r') as file:
    extended_stop_words = file.read().replace('\n', '')

In [ ]:
new_docs = []
for doc in docs:
    words = doc.split()
    new_words = []
    for word in words:
        if word in extended_stop_words:
            continue
        if word.isalpha():
            new_words.append(word)
        if word.isdigit() and (int(word) >= 1995 and int(word) <= 2016):
            new_words.append(word)
    new_docs.append(new_words)

In [ ]:
new_docs = [' '.join(w) for w in [d for d in new_docs]]

In [ ]:
docs = new_docs

In [ ]:
docs[10]

'delayed planting crop conditions lack access inputs severe dryness key messages country niño delayed start distributed rains prolonged dry conditions led delayed planting crop development cropping conditions permanent wilting districts central southern malawi reduce household access green crops consumption harvest prospects lean period extend april march percent farmers received access subsidized seeds fertilizer year farm input subsidy program fisp delays cropping season farmers access program planted recycled seeds access fertilizer inputs production prospects season reduced assistance programming cash food transfers food insecure people february funding gaps pipeline break harvest expected smaller expected green harvest expected april households face livelihood protection deficits food gaps months stressed ipc phase crisis ipc phase acute food insecurity outcomes march december food prices increased mwk monitored markets fews assessments maize prices increased levels mwk markets pr

In [ ]:
len(docs)

63609

In [ ]:
import collections

In [ ]:
all_doc = ' '.join(docs)

In [ ]:
wordcount = {}
for word in all_doc.split():
    if word not in wordcount:
        wordcount[word] = 1
    else:
        wordcount[word] += 1

In [ ]:
word_counter = collections.Counter(wordcount)
most_common = []
for word, count in word_counter.most_common(20):
    print(word, ": ", count)
    most_common.append(word)

people :  134443
food :  75762
water :  75032
health :  69812
government :  61154
relief :  60876
children :  58045
emergency :  54854
areas :  50237
support :  48184
international :  46377
disaster :  45223
response :  41785
assistance :  41493
families :  40821
country :  39634
humanitarian :  37455
local :  34900
world :  31230
new :  30820


## 2.2. Documents Padding

In [ ]:
texts_tok = Tokenizer(num_words=vocab_size)
texts_tok.fit_on_texts(docs)

labActor_tok = preprocessing.MultiLabelBinarizer()
labAgency_tok = preprocessing.MultiLabelBinarizer()
labSector_tok = preprocessing.MultiLabelBinarizer()
labPlace_tok = preprocessing.MultiLabelBinarizer()
labYear_tok = preprocessing.MultiLabelBinarizer()
labMonth_tok = preprocessing.MultiLabelBinarizer()
labReason_tok = preprocessing.MultiLabelBinarizer()
labGoal_tok = preprocessing.MultiLabelBinarizer()

labActor_tok.fit(labActor)
labAgency_tok.fit(labAgency)
labSector_tok.fit(labSector)
labPlace_tok.fit(labPlace)
labYear_tok.fit(labYear)
labMonth_tok.fit(labMonth)
labReason_tok.fit(labReason)
labGoal_tok.fit(labGoal)

MultiLabelBinarizer(classes=None, sparse_output=False)

In [ ]:
with open(f'./Tokenizer/{model_name}text.pickle', 'wb') as handle:
    pickle.dump(texts_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}actor.pickle', 'wb') as handle:
    pickle.dump(labActor_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}agency.pickle', 'wb') as handle:
    pickle.dump(labAgency_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}sector.pickle', 'wb') as handle:
    pickle.dump(labSector_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}place.pickle', 'wb') as handle:
    pickle.dump(labPlace_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}year.pickle', 'wb') as handle:
    pickle.dump(labYear_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}month.pickle', 'wb') as handle:
    pickle.dump(labMonth_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}reason.pickle', 'wb') as handle:
    pickle.dump(labReason_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'./Tokenizer/{model_name}goal.pickle', 'wb') as handle:
    pickle.dump(labGoal_tok, handle, protocol=pickle.HIGHEST_PROTOCOL)   

In [ ]:
labSector[50]

['Agriculture', 'Food and Nutrition']

In [ ]:
Find_Word = texts_tok.word_index
Find_Code = {v: k for k, v in Find_Word.items()}

In [ ]:
encoded_text = texts_tok.texts_to_sequences(docs)

encoded_actor = labActor_tok.transform(labActor)
encoded_agency = labAgency_tok.transform(labAgency)
encoded_sector = labSector_tok.transform(labSector)
encoded_place = labPlace_tok.transform(labPlace)
encoded_year = labYear_tok.transform(labYear)
encoded_month = labMonth_tok.transform(labMonth)
encoded_reason = labReason_tok.transform(labReason)
encoded_goal = labGoal_tok.transform(labGoal)

## 2.3. Text Decoder

In [ ]:
def TextDecoder(entx):
    enc = texts_tok.sequences_to_texts([entx])[0]
    tok = enc.split()
    return tok

## 2.4. Text Augmenter

In [ ]:
t_aug = Wordnet(v=True, n=True, p=0.5)

In [ ]:
def TextAugmenter(entx):
    tok = TextDecoder(entx)
    tok = ' '.join(tok)
    tok = t_aug.augment(tok)
    tok = texts_tok.texts_to_sequences([tok])
    return tok[0]

In [ ]:
aug = TextAugmenter(encoded_text[0])

print(TextDecoder(encoded_text[0]))
print(TextDecoder(aug))

print(encoded_text[0])
print(aug)

['village', 'leyte', 'island', 'typhoon', 'haiyan', 'destroyed', 'buildings', 'reduced', 'local', 'health', 'post', 'pile', 'crushed', 'concrete', 'doctors', 'bordersmédecins', 'sans', 'frontières', 'msf', 'teams', 'running', 'mobile', 'clinics', 'health', 'care', 'residents', 'including', 'psychological', 'support', 'people', 'recover', 'traumatic', 'event', 'young', 'children', 'husband', 'works', 'town', 'day', 'children', 'daughter', 'marie', 'turned', 'months', 'suffering', 'diarrhea', 'mobile', 'clinic', 'examined', 'doctor', 'medicine', 'fever', 'rash', 'brought', 'health', 'post', 'reduced', 'pile', 'rubble', 'typhoon', 'putting', 'local', 'midwife', 'daily', 'visits', 'leaving', 'villagers', 'health', 'care', 'medicines', 'pregnant', 'women', 'sick', 'people', 'choice', 'travel', 'district', 'health', 'center', 'town', 'damaged', 'typhoon', 'functioning', 'crowded', 'patients', 'region', 'typhoon', 'children', 'village', 'school', 'others', 'sheltering', 'children', 'scared', 

# 3. Model Training

## 3.1. Model Definition

### 3.1.2. Text Embedding

In [ ]:
labActor_size = len(labActor_tok.classes_)
labAgency_size = len(labAgency_tok.classes_)
labSector_size = len(labSector_tok.classes_)
labPlace_size = len(labPlace_tok.classes_)
labYear_size = len(labYear_tok.classes_)
labMonth_size = len(labMonth_tok.classes_)
labReason_size = len(labReason_tok.classes_)
labGoal_size = len(labGoal_tok.classes_)

In [ ]:
print('Number of Actors:\t', labActor_size)
print('Number of Agency:\t', labAgency_size)
print('Number of Sectors:\t', labSector_size)
print('Number of Places:\t', labPlace_size)
print('Number of Years:\t', labYear_size)
print('Number of Months:\t', labMonth_size)
print('Number of Reasons:\t', labReason_size)
print('Number of Goals:\t', labGoal_size)

Number of Actors:	 1117
Number of Agency:	 7
Number of Sectors:	 19
Number of Places:	 203
Number of Years:	 22
Number of Months:	 12
Number of Reasons:	 21
Number of Goals:	 16


### 3.1.3. Neural Network

In [ ]:
import keras
from keras import objectives
from keras.engine.topology import Layer

In [ ]:
inpText = Input(shape=(padding_len,))

x = Embedding(vocab_size, embed_size, input_length=padding_len, name='inpWeight')(inpText)
x = Flatten()(x)

x = Dense(300, activation='sigmoid')(x)

outActor = Dense(labActor_size, activation='sigmoid', name='outActor')(x)
outAgency = Dense(labAgency_size, activation='sigmoid', name='outAgency')(x)
outSector = Dense(labSector_size, activation='sigmoid', name='outSector')(x)
outPlace = Dense(labPlace_size, activation='sigmoid', name='outPlace')(x)
outYear = Dense(labYear_size, activation='sigmoid', name='outYear')(x)
outMonth = Dense(labMonth_size, activation='sigmoid', name='outMonth')(x)
outReason = Dense(labReason_size, activation='sigmoid', name='outReason')(x)
outGoal = Dense(labGoal_size, activation='sigmoid', name='outGoal')(x)
outStat = Dense(1, activation='sigmoid', name='outStat')(x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### 3.1.6. Model Compiler

In [ ]:
model = Model(inputs=(inpText),
              outputs=(outActor, outAgency, outSector, outPlace, outYear, outMonth, outReason, outGoal, outStat))

In [ ]:
opt = keras.optimizers.Adam()

In [ ]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['categorical_accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
# print(model.summary())

## 3.2. Model Optimisation

### 3.2.1. Data Preparation

In [ ]:
text_train = encoded_text[:train_split]

actor_train = encoded_actor[:train_split]
agency_train = encoded_agency[:train_split]
sector_train = encoded_sector[:train_split]
place_train = encoded_place[:train_split]
year_train = encoded_year[:train_split]
month_train = encoded_month[:train_split]
reason_train = encoded_reason[:train_split]
goal_train = encoded_goal[:train_split]
stat_train = np.array([[1]] * len(text_train))

In [ ]:
text_valid = encoded_text[train_split+1:train_limit]

actor_valid = encoded_actor[train_split+1:train_limit]
agency_valid = encoded_agency[train_split+1:train_limit]
sector_valid = encoded_sector[train_split+1:train_limit]
place_valid = encoded_place[train_split+1:train_limit]
year_valid = encoded_year[train_split+1:train_limit]
month_valid = encoded_month[train_split+1:train_limit]
reason_valid = encoded_reason[train_split+1:train_limit]
goal_valid = encoded_goal[train_split+1:train_limit]
stat_valid  = np.array([[1]] * len(text_valid))

In [ ]:
text_valid = pad_sequences(text_valid, maxlen=padding_len, padding='post', truncating='post')

### 3.2.3. Batch Generation

In [ ]:
rand_bool = 0

In [ ]:
def batch_generator(x_text, y_actor, y_agency, y_sector, y_place, y_year, y_month, y_reason, y_goal, nr_records=train_split):
    
    global rand_bool
 
    while True:
        
        batch_size = 512
 
        n_batches_per_epoch = train_split//batch_size
        rand_iter = [random.randint(0, n_batches_per_epoch) for _ in range(1)]
 
        for i in range(n_batches_per_epoch):
 
            bat_all = x_text[batch_size*i:batch_size*(i+1)]
 
            if rand_bool == 0:
                bat_text = [sent[random.randint(0, (len(sent))):][:padding_len] for sent in bat_all]
            else:
                bat_text = [sent[:padding_len] for sent in bat_all]
            
            rand_bool += 1
 
            if rand_bool >= 5 : rand_bool = 0
 
            bat_text = pad_sequences(bat_text, maxlen=padding_len, padding='post', truncating='post')
 
            bat_actor = y_actor[batch_size*i:batch_size*(i+1)]
            bat_agency = y_agency[batch_size*i:batch_size*(i+1)]
            bat_sector = y_sector[batch_size*i:batch_size*(i+1)]
            bat_place = y_place[batch_size*i:batch_size*(i+1)]
            bat_year = y_year[batch_size*i:batch_size*(i+1)]
            bat_month = y_month[batch_size*i:batch_size*(i+1)]
            bat_reason = y_reason[batch_size*i:batch_size*(i+1)]
            bat_goal = y_goal[batch_size*i:batch_size*(i+1)]
            bat_stat = np.array([[1]] * len(bat_text))
 
            if Use_Negative:
 
                if i in rand_iter:
 
                    bat_actor = [random.sample(list(x), len(x)) for x in bat_actor]
                    bat_agency = [random.sample(list(x), len(x)) for x in bat_agency]
                    bat_sector = [random.sample(list(x), len(x)) for x in bat_sector]
                    bat_place = [random.sample(list(x), len(x)) for x in bat_place]
                    bat_year = [random.sample(list(x), len(x)) for x in bat_year]
                    bat_month = [random.sample(list(x), len(x)) for x in bat_month]
                    bat_reason = [random.sample(list(x), len(x)) for x in bat_reason]
                    bat_goal = [random.sample(list(x), len(x)) for x in bat_goal]
                    bat_stat = np.array([[0]] * len(bat_text))
           
            yield [bat_text], [bat_actor, bat_agency, bat_sector, bat_place, bat_year, bat_month, bat_reason, bat_goal, bat_stat]

### 3.2.3. Model Fitting

In [ ]:
num_epoch = 500

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=0.000001)

In [ ]:
history = model.fit(batch_generator(text_train, actor_train, 
                                            agency_train, sector_train, 
                                            place_train, year_train,
                                            month_train, reason_train, goal_train, stat_train),
                    validation_data=([text_valid], [actor_valid,
                                                agency_valid, sector_valid, 
                                                place_valid, year_valid,
                                                month_valid, reason_valid, goal_valid, stat_valid]),
                    steps_per_epoch=100,
                    epochs=num_epoch,
                    verbose = 1,
                    # callbacks=[reduce_lr, early_stop],
                    # callbacks=[early_stop],
                    shuffle=True)

In [ ]:
encoded_text = pad_sequences(encoded_text, maxlen=padding_len, padding='post', truncating='post')

## 3.3. Model Evaluation

In [ ]:
np.shape(model.get_layer('inpWeight').get_weights()[0])

(10000, 300)

In [ ]:
print(model.metrics_names)

['loss', 'outActor_loss', 'outAgency_loss', 'outSector_loss', 'outPlace_loss', 'outYear_loss', 'outMonth_loss', 'outReason_loss', 'outGoal_loss', 'outStat_loss', 'outActor_categorical_accuracy', 'outAgency_categorical_accuracy', 'outSector_categorical_accuracy', 'outPlace_categorical_accuracy', 'outYear_categorical_accuracy', 'outMonth_categorical_accuracy', 'outReason_categorical_accuracy', 'outGoal_categorical_accuracy', 'outStat_categorical_accuracy']


### 3.3.1. Training Dataset Evaluation

In [ ]:
valid_encoded_text = encoded_text[:train_limit]

valid_encoded_actor = encoded_actor[:train_limit]
valid_encoded_agency = encoded_agency[:train_limit]
valid_encoded_sector = encoded_sector[:train_limit]
valid_encoded_place = encoded_place[:train_limit]
valid_encoded_year = encoded_year[:train_limit]
valid_encoded_month = encoded_month[:train_limit]
valid_encoded_reason = encoded_reason[:train_limit]
valid_encoded_goal = encoded_goal[:train_limit]
valid_encoded_stat = np.array([[1]] * len(valid_encoded_text))

In [ ]:
valid_encoded_text = pad_sequences(valid_encoded_text, maxlen=padding_len, padding='post', truncating='post')

In [ ]:
loss, _, _, _, _, _, _, _, _, _, accActor, accAgency, accSector, accPlace, accYear, accMonth, accReason, accGoal, accStat = model.evaluate(
    [valid_encoded_text],
    [valid_encoded_actor, valid_encoded_agency, valid_encoded_sector, valid_encoded_place,
     valid_encoded_year, valid_encoded_month, valid_encoded_reason, valid_encoded_goal, valid_encoded_stat], verbose=0)

In [ ]:
accAverage = (accActor + accAgency + accGoal + accSector + accPlace + accYear + accMonth + accReason) / 8

In [ ]:
print('Overall Loss is: %f' % (loss))
print('Actor Accuracy: %f' % (accActor*100))
print('Agency Accuracy: %f' % (accAgency*100))
print('SDG Accuracy: %f' % (accGoal*100))
print('Sector Accuracy: %f' % (accSector*100))
print('Place Accuracy: %f' % (accPlace*100))
print('Year Accuracy: %f' % (accYear*100))
print('Month Accuracy: %f' % (accMonth*100))
print('Reason Accuracy: %f' % (accReason*100))
print('Average Accuracy: %f' % (accAverage*100))

Overall Loss is: 0.335392
Actor Accuracy: 90.964293
Agency Accuracy: 97.093558
SDG Accuracy: 91.659951
Sector Accuracy: 63.163477
Place Accuracy: 96.156192
Year Accuracy: 93.412858
Month Accuracy: 89.812720
Reason Accuracy: 73.234814
Average Accuracy: 86.937233


### 3.3.2. Testing Dataset Evaluation

In [ ]:
test_encoded_text = encoded_text[train_limit+1:]

test_encoded_actor = encoded_actor[train_limit+1:]
test_encoded_agency = encoded_agency[train_limit+1:]
test_encoded_sector = encoded_sector[train_limit+1:]
test_encoded_place = encoded_place[train_limit+1:]
test_encoded_year = encoded_year[train_limit+1:]
test_encoded_month = encoded_month[train_limit+1:]
test_encoded_reason = encoded_reason[train_limit+1:]
test_encoded_goal = encoded_goal[train_limit+1:]
test_encoded_stat = np.array([[1]] * len(test_encoded_text))

In [ ]:
test_encoded_text = pad_sequences(test_encoded_text, maxlen=padding_len, padding='post', truncating='post')

In [ ]:
loss, _, _, _, _, _, _, _, _, _, accActor, accAgency, accSector, accPlace, accYear, accMonth, accReason, accGoal, accStat = model.evaluate(
    [test_encoded_text],
    [test_encoded_actor, test_encoded_agency, test_encoded_sector, test_encoded_place,
     test_encoded_year, test_encoded_month, test_encoded_reason, test_encoded_goal, test_encoded_stat], verbose=0)

In [ ]:
accAverage = (accActor + accAgency + accGoal + accSector + accPlace + accYear + accMonth + accReason) / 8

In [ ]:
print('Overall Loss is: %f' % (loss))
print('Actor Accuracy: %f' % (accActor*100))
print('Agency Accuracy: %f' % (accAgency*100))
print('SDG Accuracy: %f' % (accGoal*100))
print('Sector Accuracy: %f' % (accSector*100))
print('Place Accuracy: %f' % (accPlace*100))
print('Year Accuracy: %f' % (accYear*100))
print('Month Accuracy: %f' % (accMonth*100))
print('Reason Accuracy: %f' % (accReason*100))
print('Average Accuracy: %f' % (accAverage*100))

Overall Loss is: 0.946305
Actor Accuracy: 62.039149
Agency Accuracy: 86.793488
SDG Accuracy: 58.823991
Sector Accuracy: 51.796240
Place Accuracy: 81.054950
Year Accuracy: 67.541862
Month Accuracy: 50.279069
Reason Accuracy: 67.793411
Average Accuracy: 65.765270


# 4. Export Results

## 4.1. Export .H5 Model

In [ ]:
modelOutputFile = f'./Model/{model_name}SEmHus_Word_Embedding_Model.h5'
model.save(modelOutputFile)

## 4.2. Export Dictionaries

In [ ]:
list_encoded_text = f'./Dictionary/{model_name}list_encoded_text.csv'

In [ ]:
dictText = {k: v for k, v in texts_tok.word_index.items()}

In [ ]:
with open(list_encoded_text, 'w') as f:
    writer = csv.writer(f)
    for key, value in dictText.items():
        writer.writerow([key, value]) 

## 4.3. Export Embeddings

In [ ]:
word_embed_dict = f'./Dictionary/{model_name}word_embed_dict.csv'

In [ ]:
# model.layers[1].get_weights()[0]

In [ ]:
emb_word = model.get_layer('inpWeight').get_weights()[0]
dictEmbed = {w:emb_word[idx] for w, idx in dictText.items() if idx <= vocab_size-1}

In [ ]:
len(dictText.items())

153411

In [ ]:
with open(word_embed_dict, 'w') as f:
    writer = csv.writer(f)
    for key, value in dictEmbed.items():
        writer.writerow([key, np.array(value).tolist()])